In [1]:
import sys
import django
import os
file_dir = "/Users/mirbilal/Desktop/MobCommission/commissionV2/"
if file_dir not in sys.path:
    sys.path.insert(0, file_dir)

os.environ["DJANGO_SETTINGS_MODULE"] = "commissionerv2.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true" 
django.setup()
from os import path, environ
from datetime import datetime
from services.zayev.environment.market_simulator import MarketSimulator
from services.zayev.service_layer.forester import Forester
import tensorflow as tf
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.metrics import accuracy_score
from matplotlib.dates import relativedelta
from apps.environment.service_layer.data_transformer import DataTransformer
import pandas as pd
from joblib import load
import pathlib
import pytz

2024-01-01 10:06:41.061569: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [25]:
chosen_stock = 11
(current_file_path, new_file_path) = DataTransformer.get_new_file_path(chosen_stock=chosen_stock)

rf = load(pathlib.Path(current_file_path))

nwb_starting_time_step = datetime(year=2023, month=7, day=1)
now_time = pytz.utc.localize(datetime.now())

nwb_db_params = {
    'database': environ.get("POSTGRES_DB"),
    'user': environ.get("POSTGRES_USER"),
    'password': environ.get("POSTGRES_PASSWORD"),
    'host': environ.get("DB_HOST"),
    'port': environ.get("DB_PORT"),
}
nwb_env_config = {
    "db_params": nwb_db_params, 
    "max_episode_steps": 1000, 
    "the_current_time_step": nwb_starting_time_step,
    "print_output": False,
    "is_test": False,
    "test_steps": 200,
    "n_step_stocks": 5,
    "n_step_cmmdties": 5,
    "preparing": False,
    "no_of_stocks": 100,
    "no_of_cmmdts": 14,
    "is_live": True,
    "training_stock_index": chosen_stock
} 
nwb_dformer = DataTransformer(env_config=nwb_env_config)
nwb_dfs = nwb_dformer.create_input_dataframes()
nwb_market = MarketSimulator(env_config=nwb_env_config, data_frames=nwb_dfs)
nwb_market.reset()

total_init_potfolio = nwb_market.wallet_state
for initd in range(100):
    data_initd = initd+1
    total_init_potfolio = total_init_potfolio + nwb_market.stock_data[data_initd]["price_snapshot"]*nwb_market.shares_data[data_initd]
print(total_init_potfolio)
print(nwb_market.the_current_time_step - relativedelta(hours=100))

nwb_forest = Forester(market=nwb_market)
total_potfolio = nwb_market.wallet_state
latest_action = None
for ind in range(100):
    data_index = ind+1
    total_potfolio = total_potfolio + nwb_market.stock_data[data_index]["price_snapshot"]*nwb_market.shares_data[data_index]

initial_total_potfolio = total_potfolio
print(total_potfolio)
print(nwb_market.the_current_time_step, now_time)
for indx in range(200):
    if indx < 7:
        acts = np.full(100, 15)
        rand_acts = nwb_forest.act_random(acts)
        nwb_market.step(rand_acts)
    else:
        rand_acts = nwb_forest.market.get_mov_avg_actions()
        # print(rand_acts)
        acts_list = []
        iny = 1
        # rand_acts = np.full(100, 15)
        rand_acts = nwb_forest.act_random(rand_acts, use_best_stks=False)
        for an_act in rand_acts:
            acts_list.append((iny, an_act))
            iny=iny+1
        print(acts_list)
        # print(nwb_market.the_current_time_step.day, now_time.day)
        if nwb_market.the_current_time_step.day >= now_time.day and (nwb_market.the_current_time_step.month == 1):
            latest_action = acts
            break
        nwb_market.step(rand_acts)

print(nwb_market.wallet_state)

/Users/mirbilal/Desktop/MobCommission/commissionV2/apps/environment/service_layer/data_transformer.py:282: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  stcks_concatenated_df[field] = stcks_concatenated_df.groupby('index')[field].fillna(method='ffill')
/Users/mirbilal/Desktop/MobCommission/commissionV2/apps/environment/service_layer/data_transformer.py:283: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  stcks_concatenated_df[field] = stcks_concatenated_df.groupby('index')[field].fillna(method='bfill')
/Users/mirbilal/Desktop/MobCommission/commissionV2/apps/environment/service_layer/data_transformer.py:312: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  cmmdts_concatenated_df[field] = cmmdts_concatenated_df.groupby('index')[fi

[     id                                    name  index  type   symbol
 0     1            Yousaf Weaving Mills Limited      1   100     YOUW
 1     2                     Unity Foods Limited      2   100    UNITY
 2     3                     United Bank Limited      3   100      UBL
 3     4         Unilever Pakistan Foods Limited      4   100     UPFL
 4     5                    TRG Pakistan Limited      5   100      TRG
 ..  ...                                     ...    ...   ...      ...
 95   96                     Allied Bank Limited     96   100      ABL
 96   97          Air Link Communication Limited     97   100  AIRLINK
 97   98                             AGP Limited     98   100      AGP
 98   99       Adamjee Insurance Company Limited     99   100     AICL
 99  100  Abbott Laboratories (Pakistan) Limited    100   100     ABOT
 
 [100 rows x 5 columns],
     id             name  index  units symbol
 0    1             Gold      1      0   GC=F
 1    3            Wheat     

In [23]:
total_potfolio = nwb_market.wallet_state
for ind in range(100):
    data_index = ind+1
    total_potfolio = total_potfolio + nwb_market.stock_data[data_index]["price_snapshot"]*nwb_market.shares_data[data_index]
print(total_potfolio)
print(nwb_market.the_current_time_step - relativedelta(hours=100))
final_total_potfolio = total_potfolio
gain = final_total_potfolio - initial_total_potfolio
print("gain", gain)

2117138.26649289
2023-12-27 20:00:00+00:00
gain 1856570.91649289
